In [27]:
%tensorflow_version 2.x
import os
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn import metrics


## Getting data from source
df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv", na_values=['NA', '?'])
headers = df.columns.values
display(df)
print(f"output values = {df['age'].unique()}")
print("--------------------------------------------------------------------------------------------------------")

 
## Create dummies for job
dummies = pd.get_dummies(df['job'], prefix='job')
df = pd.concat([df, dummies], axis=1)
df.drop('job', axis=1, inplace=True)


## Create dummies for area
dummies = pd.get_dummies(df['area'], prefix='area')
df = pd.concat([df, dummies], axis=1)
df.drop('area', axis=1, inplace=True)


## Create dummies for product
df = pd.concat([df, pd.get_dummies(df['product'], prefix='product')], axis=1)
df.drop('product', axis=1, inplace=True)


## Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)


## PreProcessing on data set
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])
print("--------------------------------------------------------------------------------------------------------")
print("Data Sets after standardization : ")
display(df)


## Data Sets
x_df = df.columns.drop('id').drop('age')
x = df[x_df].values
y = df['age'].values


## Split training data into test and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


## Building Nueral Network
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
model.add(Dense(1))


## Training Process and EARLYSTOPPING PROCESS
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1000, verbose=1, callbacks=[monitor])


## Calculating accuracy
y_pred = model.predict(x_test)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("--------------------------------------------------------------------------------------------------------")
print(f"MSE = {MSE}")
print("--------------------------------------------------------------------------------------------------------")
print(f"RMSE = {RMSE}")
print("--------------------------------------------------------------------------------------------------------")
## plotting prediction and test
predict_list = y_pred.flatten()
plt.figure()
t = pd.DataFrame({'pred':predict_list , 'test':y_test})
print("t = ")
display(t)
print("--------------------------------------------------------------------------------------------------------")
print("t['test'].tolist() = ")
print(t['test'].tolist())
print("--------------------------------------------------------------------------------------------------------")
print("t['pred'].tolist() = ")
print(t['pred'].tolist())
plt.plot(t['test'].tolist(), label='expected')
plt.plot(t['pred'].tolist(), label='prediction')
plt.legend()
plt.show()

id job area   income  ...  pop_dense  retail_dense     crime  product
0        1  vv    c  50876.0  ...   0.885827      0.492126  0.071100        b
1        2  kd    c  60369.0  ...   0.874016      0.342520  0.400809        c
2        3  pe    c  55126.0  ...   0.944882      0.724409  0.207723        b
3        4  11    c  51690.0  ...   0.889764      0.444882  0.361216        b
4        5  kl    d  28347.0  ...   0.744094      0.661417  0.068033        a
...    ...  ..  ...      ...  ...        ...           ...       ...      ...
1995  1996  vv    c  51017.0  ...   0.881890      0.744094  0.104838        b
1996  1997  kl    d  26576.0  ...   0.944882      0.877953  0.063851        a
1997  1998  kl    d  28595.0  ...   0.759843      0.744094  0.098703        f
1998  1999  qp    c  67949.0  ...   0.909449      0.598425  0.117803        c
1999  2000  pe    c  61467.0  ...   0.925197      0.539370  0.451973        c

[2000 rows x 14 columns]

output values = [49 51 44 50 38 43 39 45 42 48 46 47 41 40 37 35 32 33 36 52 31 34 29 30]
--------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------
Data Sets after standardization : 


id    income    aspect  ...  product_e  product_f  product_g
0        1 -0.607550 -0.664918  ...          0          0          0
1        2  0.338053 -0.207748  ...          0          0          0
2        3 -0.184205  1.127906  ...          0          0          0
3        4 -0.526467 -0.440815  ...          0          0          0
4        5 -2.851675  1.638861  ...          0          0          0
...    ...       ...       ...  ...        ...        ...        ...
1995  1996 -0.593504  1.414758  ...          0          0          0
1996  1997 -3.028085  1.011372  ...          0          0          0
1997  1998 -2.826971  1.513363  ...          0          1          0
1998  1999  1.093101 -1.274478  ...          0          0          0
1999  2000  0.447425 -0.351174  ...          0          0          0

[2000 rows x 55 columns]

Train on 1500 samples, validate on 500 samples
Epoch 1/1000
1500/1500 [==============================] - 1s 405us/sample - loss: 2067.5865 - accuracy: 0.0000e+00 - val_loss: 1960.3802 - val_accuracy: 0.0000e+00
Epoch 2/1000
1500/1500 [==============================] - 0s 89us/sample - loss: 1779.3517 - accuracy: 0.0000e+00 - val_loss: 1480.0143 - val_accuracy: 0.0000e+00
Epoch 3/1000
1500/1500 [==============================] - 0s 96us/sample - loss: 1085.6552 - accuracy: 0.0000e+00 - val_loss: 646.0960 - val_accuracy: 0.0000e+00
Epoch 4/1000
1500/1500 [==============================] - 0s 106us/sample - loss: 398.1814 - accuracy: 0.0000e+00 - val_loss: 253.7795 - val_accuracy: 0.0000e+00
Epoch 5/1000
1500/1500 [==============================] - 0s 87us/sample - loss: 240.8640 - accuracy: 0.0000e+00 - val_loss: 223.4240 - val_accuracy: 0.0000e+00
Epoch 6/1000
1500/1500 [==============================] - 0s 88us/sample - loss: 218.9337 - accuracy: 0.0000e+00 - val_loss: 205.8155 - val_a

pred  test
0    49.813641    50
1    36.969299    36
2    48.410519    48
3    48.549477    48
4    44.846577    45
..         ...   ...
495  48.073135    48
496  49.856247    50
497  37.769508    37
498  45.326859    46
499  49.543861    50

[500 rows x 2 columns]

--------------------------------------------------------------------------------------------------------
t['test'].tolist() = 
[50, 36, 48, 48, 45, 44, 45, 43, 47, 49, 42, 50, 48, 49, 48, 48, 41, 43, 45, 47, 50, 50, 46, 36, 36, 32, 49, 40, 47, 49, 45, 46, 50, 49, 49, 41, 40, 49, 49, 50, 47, 48, 50, 48, 46, 44, 48, 48, 47, 37, 46, 45, 46, 42, 48, 50, 48, 39, 48, 49, 47, 47, 45, 44, 46, 49, 48, 39, 44, 47, 49, 41, 51, 39, 45, 48, 44, 49, 49, 47, 47, 39, 49, 41, 48, 50, 47, 49, 48, 45, 43, 48, 43, 47, 49, 46, 50, 51, 49, 46, 50, 46, 43, 46, 50, 47, 41, 46, 47, 48, 44, 45, 49, 47, 46, 47, 48, 41, 45, 47, 49, 41, 45, 43, 45, 39, 50, 45, 49, 49, 49, 49, 40, 48, 50, 51, 46, 47, 45, 43, 50, 49, 47, 49, 48, 47, 46, 45, 48, 47, 39, 47, 39, 49, 41, 43, 48, 45, 40, 48, 41, 48, 45, 50, 51, 44, 47, 47, 45, 47, 47, 44, 50, 45, 50, 44, 46, 46, 43, 48, 47, 47, 43, 47, 50, 50, 43, 43, 45, 50, 41, 36, 44, 49, 50, 41, 44, 47, 47, 47, 46, 46, 32, 50, 45, 49, 45, 45, 44, 47, 50, 45, 51, 46, 45, 49, 48, 41, 